In [132]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re
import numpy as np

df=pd.read_csv("milano_dati_booking.csv")
df['prezzo']= df['prezzo'].str.replace(".","")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale']= df['prezzo_iniziale'].str.replace(".","")
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))
try: df['credito_guadagnato'] = df['credito_guadagnato'].str.extract(r'(\d+)').astype(float)
except: pass
df['offerte'].unique()


array(['Offerta a tempo limitato ', nan, 'Prezzo solo su mobile ',
       'Offerta Inizio 2024 ',
       'Prezzo solo su mobile Offerta SuperSegreta ',
       'Offerta SuperSegreta ', "Taxi gratuito dall'aeroporto ",
       "Taxi gratuito dall'aeroporto Offerta SuperSegreta ",
       'Booking.com paga ', 'Offerta Inizio 2024 Booking.com paga ',
       'Prezzo solo su mobile Booking.com paga ',
       "Taxi gratuito dall'aeroporto Booking.com paga ",
       'Offerta a tempo limitato Booking.com paga ',
       'Prezzo solo su mobile Offerta SuperSegreta Booking.com paga ',
       'Offerta SuperSegreta Booking.com paga ',
       "Taxi gratuito dall'aeroporto Offerta Inizio 2024 ",
       "Taxi gratuito dall'aeroporto Offerta a tempo limitato "],
      dtype=object)

In [133]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)
df['partner_preferiti'] = df['partner_preferiti'].fillna("NaN")
df['partner_preferiti'].unique()

array(['NaN', 'preferred', 'preferred_plus'], dtype=object)

In [85]:
# Visualizzo OS in base a username
df_user = df[df['username']=='pantilaura56']
lst = df_user['TIMESTAMP'].unique()
os = []
for i in lst:
    df3 = df_user[df_user['TIMESTAMP']==i]
    os.append(df3['os'].unique()[0])
os

['Android',
 'Android',
 'Windows',
 'Windows',
 'Linux',
 'Linux',
 'MacOS',
 'MacOS']

In [86]:
# Visualizzo OS in base a username
df_user = df[df['username']=='sunnytraveler']
lst = df_user['TIMESTAMP'].unique()
os = []
for i in lst:
    df3 = df_user[df_user['TIMESTAMP']==i]
    os.append(df3['os'].unique()[0])
os

['Android',
 'Windows',
 'Linux',
 'Android',
 'Windows',
 'Linux',
 'MacOS',
 'MacOS']

In [87]:
# Visualizzo OS in base a username
df_user = df[df['username']=='marcofantile']
lst = df_user['TIMESTAMP'].unique()
os = []
for i in lst:
    df3 = df_user[df_user['TIMESTAMP']==i]
    os.append(df3['os'].unique()[0])
os 

['Linux',
 'Android',
 'Android',
 'MacOS',
 'Windows',
 'Windows',
 'Linux',
 'MacOS']

In [135]:
# Visualizza lista ricerche per orario
df['TIMESTAMP'].unique()

array(['03-07-2024---11:52', '03-07-2024---12:08', '03-07-2024---12:34',
       '03-07-2024---12:46', '03-07-2024---13:02', '03-07-2024---13:14',
       '03-07-2024---13:25', '03-07-2024---13:40', '03-07-2024---14:00',
       '03-07-2024---14:20', '03-07-2024---14:38', '03-07-2024---14:49',
       '03-07-2024---14:59', '03-07-2024---15:08', '03-07-2024---15:20',
       '03-07-2024---15:32', '03-07-2024---15:43', '03-07-2024---15:55',
       '03-07-2024---16:09', '03-07-2024---16:51', '03-07-2024---17:03',
       '03-07-2024---17:21', '03-07-2024---17:32', '03-07-2024---17:51'],
      dtype=object)

In [136]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[12]
orario2 = df["TIMESTAMP"].unique()[9]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [137]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[3:4] +df_list[5:6] +df_list[7:]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
initial_len = len(merged_df)
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)      



In [138]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y",'offerte_x','offerte_y']]


merged_df.columns


Index(['nome_hotel', 'prezzo_iniziale', 'prezzo_x', 'credito_guadagnato_x',
       'stanza_x', 'città_x', 'data', 'partner_preferiti_x', 'evidenza_x',
       'numero_recensioni_x', 'distanza_centro_x', 'offerte_x',
       'senza_pagamento_anticipato_x', 'cancellazione_gratuita_x', 'os_x',
       'username_x', 'user_agent_type_x', 'TIMESTAMP_x',
       'offerta_solo_mobile_x', 'offerta_super_segreta_x',
       'offerta_inizio_2024_x', 'prezzo_y', 'credito_guadagnato_y', 'stanza_y',
       'città_y', 'partner_preferiti_y', 'evidenza_y', 'numero_recensioni_y',
       'distanza_centro_y', 'offerte_y', 'senza_pagamento_anticipato_y',
       'cancellazione_gratuita_y', 'os_y', 'username_y', 'user_agent_type_y',
       'TIMESTAMP_y', 'offerta_solo_mobile_y', 'offerta_super_segreta_y',
       'offerta_inizio_2024_y', 'punteggio', 'stelle', 'stelle_booking',
       'novita', 'genius', 'colazione_inclusa', 'booking_paga',
       'offerta_tempo_limitato'],
      dtype='object')

In [140]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
merged_df = merged_df.sort_values(by='differenza_prezzo',ascending= False)

merged_df[['nome_hotel','prezzo_x','prezzo_y','differenza_prezzo','os_x','os_y']]

,nome_hotel,prezzo_x,prezzo_y,differenza_prezzo,os_x,os_y
778,Heart Milan Apartments Sforza Duomo,500,451,49,Linux,Android
253,Santa Sofia Home,268,224,44,Linux,Android
628,[FieraMilano-San Siro] Exclusive Milanese House,216,174,42,Linux,Android
630,Room Mate Giulia,422,380,42,Linux,Android
468,Vmaison Brera Milano,416,375,41,Linux,Android
264,Magna Pars l'Hotel à Parfum,370,334,36,Linux,Android
225,Suite boutique Moscova,353,318,35,Linux,Android
484,Duomo Rooms,293,258,35,Linux,Android
749,Ai Suma Hotel,341,307,34,Linux,Android
517,Petit Palais Hotel De Charme,321,289,32,Linux,Android


In [22]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
#df_mobile = merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])]
#len(df_mobile)
len(merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])])

659

In [ ]:
# Elimino righe in cui il mobile non ha la stanza
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)